In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
import cv2
from torchvision import transforms
model.eval()

In [ ]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
pred =newmodel(input_tensor)

In [ ]:
pred.shape

In [ ]:
image = cv2.imread("sample_images/a0676-kme_609_c.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
input = cv2.resize(image, (64, 64)) / 255.0
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input).unsqueeze(0).type(torch.float32)
target =newmodel(input_tensor)

In [1]:
from envs.env_dataloader import create_dataloaders
import torch
train,test = create_dataloaders()

Encoding testing data ...


  0%|          | 0/4 [00:00<?, ?it/s]

finished...
Encoding training data ...


  0%|          | 0/36 [00:00<?, ?it/s]

finished...


In [2]:
f=train.dataset.encoded_source

In [3]:
from envs.new_edit_photo import PhotoEditor

editor = PhotoEditor()
parameters = torch.tensor([0.125, 0.125, 0.375, 0.125, 0., 0.0625, 0.9375, 0.375, 0.0625, 0., 0.125, 0.125])
parameters = torch.stack([parameters for i in range(64)])

In [45]:
for i,j,k,l in train:
    # print(i.dtype)
    enhanced = i.permute(0,2,3,1)/255.0
    # print(enhanced.dtype)
    enhanced = editor(enhanced,parameters)
    enhanced = enhanced.permute(0,3,1,2)
    enhanced =torch.flatten(enhanced,start_dim=1, end_dim=-1)

    target = torch.flatten(j,start_dim=1, end_dim=-1)/255.0
    # print(enhanced.dtype)
    rmse = enhanced-target
    rmse = torch.pow(rmse,2).mean(1)
    rmse = torch.sqrt(rmse)
    break

print(rmse.shape)

torch.Size([64])


In [75]:
torch.tensor([i for i in range(10)])

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [79]:
a = torch.tensor([1,2,3,4,7])
print((a>3))

tensor([False, False, False,  True,  True])


In [77]:
a[torch.tensor([1])]

tensor([2])

In [74]:
[2,3][True,False]

<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
/tmp/ipykernel_13011/3387127371.py:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [2,3][True,False]


TypeError: list indices must be integers or slices, not tuple

In [40]:
torch.rand(3,64,64)[[1,2],...].shape

torch.Size([2, 64, 64])

In [71]:
f= [i for i in range(10)]
f[f>3]

TypeError: '>' not supported between instances of 'list' and 'int'

In [69]:
~[True,True]

TypeError: bad operand type for unary ~: 'list'

In [61]:
torch.rand(2,64)[[True,True],:].shape

torch.Size([0, 64])